# Arabic Diacritization using RoBerta Transformers implementation

In [ ]:
# THIS PART IS ONLY FOR COLAB
! pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"peteratef","key":"1b4b429039c7e2e3879faea35cfc725b"}'}

In [ ]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [ ]:
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d hamzaabbad/tashkeela-processed-fully-diacritized-arabic-text

100% 132M/132M [00:01<00:00, 104MB/s] 



In [ ]:
!mkdir /AraBertA
!unzip tashkeela-processed-fully-diacritized-arabic-text.zip -d ./AraBertA

Archive:  tashkeela-processed-fully-diacritized-arabic-text.zip
  inflating: ./AraBertA/tashkeela_test/tashkeela_test_001.txt  
  inflating: ./AraBertA/tashkeela_test/tashkeela_test_002.txt  
  inflating: ./AraBertA/tashkeela_test/tashkeela_test_003.txt  
  inflating: ./AraBertA/tashkeela_test/tashkeela_test_004.txt  
  inflating: ./AraBertA/tashkeela_test/tashkeela_test_005.txt  
  inflating: ./AraBertA/tashkeela_test/tashkeela_test_006.txt  
  inflating: ./AraBertA/tashkeela_test/tashkeela_test_007.txt  
  inflating: ./AraBertA/tashkeela_test/tashkeela_test_008.txt  
  inflating: ./AraBertA/tashkeela_test/tashkeela_test_009.txt  
  inflating: ./AraBertA/tashkeela_test/tashkeela_test_010.txt  
  inflating: ./AraBertA/tashkeela_test/tashkeela_test_011.txt  
  inflating: ./AraBertA/tashkeela_test/tashkeela_test_012.txt  
  inflating: ./AraBertA/tashkeela_test/tashkeela_test_013.txt  
  inflating: ./AraBertA/tashkeela_test/tashkeela_test_014.txt  
  inflating: ./AraBertA/tashkeela_test/t

## First we need a tokenizer

In [ ]:
!nvidia-smi

Sat Jul 17 18:24:04 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive

drive.mount('gdrive')

Mounted at gdrive


In [ ]:
!mkdir ./AraBertA/model

In [ ]:
!cp -r ./gdrive/MyDrive/Models/RoBertA/model_only_diac/  ./AraBertA/model

In [ ]:
!pip3 install tokenizers
from tokenizers import ByteLevelBPETokenizer
tokenizer = ByteLevelBPETokenizer()

### Getting all the arabic combinations

In [ ]:
arabic_letters = ['أ','ة','إ','ؤ','آ','ا','ب','ت', 'ث','ج','ح','خ','د','ذ','ر','ز','س','ش','ص','ض','ط','ظ','ع','غ','ف','ق','ك','ل','م','ن','ه','و','ي','ئ','ئ','ء']
arabic_diac = ["َ","ً","ِ","ٍ","ُ","ٌ","ْ","َّ","ِّ","ُّ"]

arabic_combinations= [];
for letter in arabic_letters:
    for diac in arabic_diac:
        arabic_combinations.append(letter+diac)
    

### Adding the tokens 
(Note the tokenizer won't save the tokens/ this is a tokenizer bug at the time of writing this code)

In [ ]:
tokenizer.add_tokens(arabic_combinations)
tokenizer.add_tokens(arabic_letters)
tokenizer.add_special_tokens([
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])


5

In [ ]:
tokenizer.save_model('./AraBertA/')

['./AraBertA/vocab.json', './AraBertA/merges.txt']

### Testing the tokenizer

In [ ]:
tokenizer.encode("الَسَلَام").ids

[355, 270, 160, 270, 355, 378]

Looks Like it's working great

## Now for the Roberta Model

### Making the tokenizer a RobertaTokenizer

In [ ]:
!pip install transformers
from transformers import RobertaTokenizerFast

roberta_tokenizer = RobertaTokenizerFast.from_pretrained('./AraBertA/')

roberta_tokenizer.add_tokens(arabic_diac)
roberta_tokenizer.add_tokens(" ")
roberta_tokenizer.add_tokens(arabic_letters)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


35

### Creating a new instance from the Roberta Model

In [ ]:
from transformers import RobertaConfig,RobertaForMaskedLM

config = RobertaConfig(
    vocab_size=len(roberta_tokenizer.get_vocab()),
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

model = RobertaForMaskedLM.from_pretrained('./AraBertA/model/model_only_diac')

In [ ]:
model.config

RobertaConfig {
  "_name_or_path": "./AraBertA/model/model_only_diac",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.8.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 51
}

In [ ]:
roberta_tokenizer.tokenize('السَلام عَليكُم')

['ا', 'ل', 'س', 'َ', 'ل', 'ا', 'م', ' ', 'ع', 'َ', 'ل', 'ي', 'ك', 'ُ', 'م']

Looks Like it's working as well

## Preparing the dataset

### Importing all the files

In [ ]:
from pathlib import Path

paths_train = [str(x) for x in Path("./AraBertA/tashkeela_train/").glob("*.txt")]
paths_eval = [str(x) for x in Path("./AraBertA/tashkeela_val/").glob("*.txt")]
paths_test = [str(x) for x in Path("./AraBertA/tashkeela_test/").glob("*.txt")]

### The Datasets library is a time-saving library from hugging face and it does it's job pretty well in keeping everything organized

In [ ]:
!pip install datasets
from datasets import load_dataset,DatasetDict

train_dataset = load_dataset('text',data_files=paths_train, split='train')
eval_dataset = load_dataset('text',data_files=paths_eval, split='train')
test_dataset = load_dataset('text',data_files=paths_test, split='train')

     |████████████████████████████████| 266kB 3.9MB/s 
     |████████████████████████████████| 122kB 10.3MB/s 
     |████████████████████████████████| 245kB 9.4MB/s 


Using custom data configuration default-1f5fa8cbbc2d2b5b


Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-1f5fa8cbbc2d2b5b/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5. Subsequent calls will reuse this data.


Using custom data configuration default-8f46f7ef9e7234ec


Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-8f46f7ef9e7234ec/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5. Subsequent calls will reuse this data.


Using custom data configuration default-d2ddc4494a8cc45d


Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-d2ddc4494a8cc45d/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5. Subsequent calls will reuse this data.


In [ ]:
# Forming the final DataSetDict
ds = DatasetDict({
    'train': train_dataset,
    'test': test_dataset,
    'eval': eval_dataset})

In [ ]:
ds['train']

Dataset({
    features: ['text'],
    num_rows: 2790000
})

In [ ]:
roberta_tokenizer.decode(15)

' '

## In this next part we will try to augment the data by creating different variants of each sentence with random diacritics stripped from each sentence

### Augmentiong the dataset
Careful as this cell takes a very long time to run

In [ ]:
def encode(examples):
    return roberta_tokenizer(examples["text"], padding="max_length", truncation=True, max_length=256)


encoded_train_dataset = ds['train'].map(encode, batched = True);
encoded_eval_dataset = ds['eval'].map(encode, batched = True);
encoded_test_dataset = ds['test'].map(encode, batched = True);

In [ ]:
encoded_train_dataset.set_format('torch', columns=['input_ids','attention_mask'])
encoded_eval_dataset.set_format('torch', columns=['input_ids','attention_mask'])
encoded_test_dataset.set_format('torch', columns=['input_ids','attention_mask'])

 ### Now we need to explicity specify for the data collator not to mask any letters, only the diacritics

In [ ]:
import random
import warnings
from dataclasses import dataclass
from typing import Any, Callable, Dict, List, NewType, Optional, Tuple, Union

import torch
from torch.nn.utils.rnn import pad_sequence
from transformers.file_utils import PaddingStrategy
from transformers.modeling_utils import PreTrainedModel
from transformers.tokenization_utils_base import BatchEncoding, PreTrainedTokenizerBase

@dataclass
class DataCollatorForLanguageModeling:
    """
    Data collator used for language modeling. Inputs are dynamically padded to the maximum length of a batch if they
    are not all of the same length.

    Args:
        tokenizer (:class:`~transformers.PreTrainedTokenizer` or :class:`~transformers.PreTrainedTokenizerFast`):
            The tokenizer used for encoding the data.
        mlm (:obj:`bool`, `optional`, defaults to :obj:`True`):
            Whether or not to use masked language modeling. If set to :obj:`False`, the labels are the same as the
            inputs with the padding tokens ignored (by setting them to -100). Otherwise, the labels are -100 for
            non-masked tokens and the value to predict for the masked token.
        mlm_probability (:obj:`float`, `optional`, defaults to 0.15):
            The probability with which to (randomly) mask tokens in the input, when :obj:`mlm` is set to :obj:`True`.
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.

    .. note::

        For best performance, this data collator should be used with a dataset having items that are dictionaries or
        BatchEncoding, with the :obj:`"special_tokens_mask"` key, as returned by a
        :class:`~transformers.PreTrainedTokenizer` or a :class:`~transformers.PreTrainedTokenizerFast` with the
        argument :obj:`return_special_tokens_mask=True`.
    """

    tokenizer: PreTrainedTokenizerBase
    mlm: bool = True
    mlm_probability: float = 0.15
    pad_to_multiple_of: Optional[int] = None

    def __post_init__(self):
        if self.mlm and self.tokenizer.mask_token is None:
            raise ValueError(
                "This tokenizer does not have a mask token which is necessary for masked language modeling. "
                "You should pass `mlm=False` to train on causal language modeling instead."
            )

    def __call__(
        self, examples: List[Union[List[int], torch.Tensor, Dict[str, torch.Tensor]]]
    ) -> Dict[str, torch.Tensor]:
        # Handle dict or lists with proper padding and conversion to tensor.
        if isinstance(examples[0], (dict, BatchEncoding)):
            batch = self.tokenizer.pad(examples, return_tensors="pt", pad_to_multiple_of=self.pad_to_multiple_of)
        else:
            batch = {"input_ids": _collate_batch(examples, self.tokenizer, pad_to_multiple_of=self.pad_to_multiple_of)}

        # If special token mask has been preprocessed, pop it from the dict.
        special_tokens_mask = batch.pop("special_tokens_mask", None)
        if self.mlm:
            batch["input_ids"], batch["labels"] = self.mask_tokens(
                batch["input_ids"], special_tokens_mask=special_tokens_mask
            )
        else:
            labels = batch["input_ids"].clone()
            if self.tokenizer.pad_token_id is not None:
                labels[labels == self.tokenizer.pad_token_id] = -100
            batch["labels"] = labels
        return batch

    def mask_tokens(
        self, inputs: torch.Tensor, special_tokens_mask: Optional[torch.Tensor] = None
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        """
        Prepare masked tokens inputs/labels for masked language modeling: 80% MASK, 10% random, 10% original.
        """
        labels = inputs.clone()
        # We sample a few tokens in each sequence for MLM training (with probability `self.mlm_probability`)
        probability_matrix = torch.full(labels.shape, self.mlm_probability)
        if special_tokens_mask is None:
            special_tokens_mask = [
                self.tokenizer.get_special_tokens_mask(val, already_has_special_tokens=True) for val in labels.tolist()
            ]
            my_mask = inputs> 15
            special_tokens_mask = torch.tensor(special_tokens_mask, dtype=torch.bool) | my_mask
            
        else:
            special_tokens_mask = special_tokens_mask.bool()

        probability_matrix.masked_fill_(special_tokens_mask, value=0.0)
        masked_indices = torch.bernoulli(probability_matrix).bool()
        labels[~masked_indices] = -100  # We only compute loss on masked tokens

        # 80% of the time, we replace masked input tokens with tokenizer.mask_token ([MASK])
        indices_replaced = torch.bernoulli(torch.full(labels.shape, 0.8)).bool() & masked_indices
        inputs[indices_replaced] = self.tokenizer.convert_tokens_to_ids(self.tokenizer.mask_token)

        # 10% of the time, we replace masked input tokens with random word
        indices_random = torch.bernoulli(torch.full(labels.shape, 0.5)).bool() & masked_indices & ~indices_replaced
        random_words = torch.randint(len(self.tokenizer), labels.shape, dtype=torch.long)
        inputs[indices_random] = random_words[indices_random]

        # The rest of the time (10% of the time) we keep the masked input tokens unchanged
        return inputs, labels


In [ ]:
data_collator = DataCollatorForLanguageModeling(
tokenizer=roberta_tokenizer, mlm=True, mlm_probability=0.45)

## Defining the Trainer

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir = "./AraBertA/model",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size= 88,
    per_device_eval_batch_size= 20,
    logging_steps=10,
    save_steps=5,
    save_total_limit=2,
)
trainer = Trainer(
    model=model, args=training_args, data_collator=data_collator ,train_dataset=encoded_train_dataset, eval_dataset= encoded_eval_dataset)

### Now it's finally time to train

In [ ]:
%%time

from torch import cuda
cuda.empty_cache()
trainer.train()

***** Running training *****
  Num examples = 2790000
  Num Epochs = 1
  Instantaneous batch size per device = 88
  Total train batch size (w. parallel, distributed & accumulation) = 88
  Gradient Accumulation steps = 1
  Total optimization steps = 31705


Step,Training Loss
10,0.100100
20,0.096000
30,0.100500
40,0.099900
50,0.106600
60,0.101600
70,0.107300
80,0.110500
90,0.106200
100,0.096700


Streaming output truncated to the last 5000 lines.
Saving model checkpoint to ./AraBertA/model/checkpoint-25345
Configuration saved in ./AraBertA/model/checkpoint-25345/config.json
Model weights saved in ./AraBertA/model/checkpoint-25345/pytorch_model.bin
Deleting older checkpoint [AraBertA/model/checkpoint-25335] due to args.save_total_limit
Saving model checkpoint to ./AraBertA/model/checkpoint-25350
Configuration saved in ./AraBertA/model/checkpoint-25350/config.json
Model weights saved in ./AraBertA/model/checkpoint-25350/pytorch_model.bin
Deleting older checkpoint [AraBertA/model/checkpoint-25340] due to args.save_total_limit
Saving model checkpoint to ./AraBertA/model/checkpoint-25355
Configuration saved in ./AraBertA/model/checkpoint-25355/config.json
Model weights saved in ./AraBertA/model/checkpoint-25355/pytorch_model.bin
Deleting older checkpoint [AraBertA/model/checkpoint-25345] due to args.save_total_limit
Saving model checkpoint to ./AraBertA/model/checkpoint-25360
Config

Step,Training Loss
10,0.100100
20,0.096000
30,0.100500
40,0.099900
50,0.106600
60,0.101600
70,0.107300
80,0.110500
90,0.106200
100,0.096700


Saving model checkpoint to ./AraBertA/model/checkpoint-31595
Configuration saved in ./AraBertA/model/checkpoint-31595/config.json
Model weights saved in ./AraBertA/model/checkpoint-31595/pytorch_model.bin
Deleting older checkpoint [AraBertA/model/checkpoint-31585] due to args.save_total_limit
Saving model checkpoint to ./AraBertA/model/checkpoint-31600
Configuration saved in ./AraBertA/model/checkpoint-31600/config.json
Model weights saved in ./AraBertA/model/checkpoint-31600/pytorch_model.bin
Deleting older checkpoint [AraBertA/model/checkpoint-31590] due to args.save_total_limit
Saving model checkpoint to ./AraBertA/model/checkpoint-31605
Configuration saved in ./AraBertA/model/checkpoint-31605/config.json
Model weights saved in ./AraBertA/model/checkpoint-31605/pytorch_model.bin
Deleting older checkpoint [AraBertA/model/checkpoint-31595] due to args.save_total_limit
Saving model checkpoint to ./AraBertA/model/checkpoint-31610
Configuration saved in ./AraBertA/model/checkpoint-31610/

CPU times: user 10h 28min 14s, sys: 1h 8min 6s, total: 11h 36min 20s
Wall time: 13h 43min 4s


In [ ]:
fill_mask = pipeline(
    "fill-mask",
    model="./AraBertA/model/checkpoint-31705/",
    tokenizer=roberta_tokenizer
)

loading configuration file ./AraBertA/model/checkpoint-31705/config.json
Model config RobertaConfig {
  "_name_or_path": "./AraBertA/model/model_only_diac",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.8.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 51
}

loading configuration file ./AraBertA/model/checkpoint-31705/config.json
Model config RobertaConfig {
  "_name_or_path": "./AraBertA/model/model_only_diac",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_

In [ ]:
ds['test']['text'][0]

'فَلَمّا كَانَ صُلْحُ الحُدَيْبِيَةِ بَيْنَ رَسُولِ اللّهِ صَلّى اللّهُ عَلَيْهِ وَسَلّمَ وَبَيْنَ قُرَيْشٍ ،'

In [ ]:
fill_mask('يَكتب<mask> الرجل الدرس')

[{'score': 0.7895705699920654,
  'sequence': 'يَكتبُ الرجل الدرس',
  'token': 9,
  'token_str': 'ُ'},
 {'score': 0.08128496259450912,
  'sequence': 'يَكتبِ الرجل الدرس',
  'token': 7,
  'token_str': 'ِ'},
 {'score': 0.06709810346364975,
  'sequence': 'يَكتبْ الرجل الدرس',
  'token': 11,
  'token_str': 'ْ'},
 {'score': 0.0611436702311039,
  'sequence': 'يَكتبَ الرجل الدرس',
  'token': 5,
  'token_str': 'َ'},
 {'score': 0.0003842785954475403,
  'sequence': 'يَكتب  الرجل الدرس',
  'token': 15,
  'token_str': ' '}]